In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import  numpy as np
print(tf.__version__)

2.0.0-rc1


In [3]:
class Regressor(keras.layers.Layer):
    def __init__(self):
        super(Regressor, self).__init__()
        # here must specify shape instead of tensor !
        # [dim_in, dim_out]
        self.w = self.add_variable('w', [13, 1])
        # [dim_out]
        self.b = self.add_variable('b', [1])

        print(self.w.shape, self.b.shape)
        print(type(self.w), tf.is_tensor(self.w), self.w.name)
        print(type(self.b), tf.is_tensor(self.b), self.b.name)


    def call(self, x):
        return tf.matmul(x, self.w) + self.b

In [4]:
(x_train, y_train), (x_val, y_val) = keras.datasets.boston_housing.load_data()
x_train, x_val = x_train.astype(np.float32), x_val.astype(np.float32)
# (404, 13) (404,) (102, 13) (102,)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

57344/57026 [==============================] - 2s 27us/step
(404, 13) (404,) (102, 13) (102,)


In [5]:
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
dataset_val = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(102)

In [8]:
tf.random.set_seed(22)
np.random.seed(22)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')


model = Regressor()
criteon = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=1e-2)

for epoch in range(200):
    for step, (x, y) in enumerate(dataset_train):
        with tf.GradientTape() as tape:
            # [b, 1]
            y_hat = model(x)
            # [b]
            y_hat = tf.squeeze(y_hat, axis=1)
            # [b] vs [b]
            train_loss = criteon(y, y_hat)

        grads = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if epoch % 10 == 0:
        for x, y in dataset_val:
            # [b, 1]
            y_hat = model(x)
            # [b]
            y_hat = tf.squeeze(y_hat, axis=1)
            # [b] vs [b]
            val_loss = criteon(y, y_hat)
            print("epoch={:0>3d}\ttrain_loss={:.6f}\tval_loss={:.6f}".format(epoch,train_loss,val_loss))


(13, 1) (1,)
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True w:0
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True b:0
epoch=000	train_loss=39695.023438	val_loss=36500.550781
epoch=010	train_loss=42.267826	val_loss=124.677483
epoch=020	train_loss=38.738976	val_loss=120.318687
epoch=030	train_loss=35.925831	val_loss=114.442184
epoch=040	train_loss=32.724255	val_loss=107.796265
epoch=050	train_loss=29.603855	val_loss=101.227325
epoch=060	train_loss=26.820536	val_loss=95.070663
epoch=070	train_loss=24.471895	val_loss=89.491501
epoch=080	train_loss=22.579117	val_loss=84.549286
epoch=090	train_loss=21.120289	val_loss=80.235916
epoch=100	train_loss=20.048559	val_loss=76.501846
epoch=110	train_loss=19.305262	val_loss=73.276276
epoch=120	train_loss=18.829227	val_loss=70.481697
epoch=130	train_loss=18.562893	val_loss=68.043526
epoch=140	train_loss=18.455795	val_loss=65.895645
epoch=150	train_loss=18.465954	val_loss=63.982647
epoch=160	train_